In [1]:
%pip install --upgrade --quiet  lark langchain-chroma

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
rtfde 0.1.2 requires lark~=1.1.8, but you have lark 1.2.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
import os

# Khởi tạo embedding và mô hình
HF_EMBEDDING = HuggingFaceEmbeddings(model_name="keepitreal/vietnamese-sbert")
llm = ChatGroq(model_name="llama3-70b-8192", temperature=0.1, api_key=os.getenv('llm_api_1'))

# Định nghĩa các tài liệu với nội dung đã được dịch sang tiếng Việt
docs = [
    Document(
        page_content="Một nhóm các nhà khoa học mang khủng long trở lại và mọi thứ trở nên hỗn loạn",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio bị lạc trong một giấc mơ trong một giấc mơ trong một giấc mơ trong một ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="Một nhà tâm lý học / thám tử bị lạc trong một loạt các giấc mơ trong những giấc mơ và Inception đã tái sử dụng ý tưởng này",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="Một nhóm phụ nữ có kích thước bình thường rất lành mạnh và một số đàn ông thầm mơ về họ",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Các món đồ chơi trở nên sống động và có một khoảng thời gian vui vẻ khi làm vậy",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Ba người đàn ông bước vào Khu vực, ba người đàn ông bước ra khỏi Khu vực",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]

# Tạo vectorstore từ các tài liệu đã dịch
vectorstore = Chroma.from_documents(docs, HF_EMBEDDING)


/home/justtuananh/miniconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

# Thông tin về các trường metadata
metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="Thể loại của bộ phim. Một trong các thể loại ['khoa học viễn tưởng', 'hài', 'drama', 'hình sự', 'lãng mạn', 'hành động', 'hoạt hình']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="Năm phát hành bộ phim",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="Tên của đạo diễn bộ phim",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="Đánh giá từ 1-10 cho bộ phim", type="float"
    ),
]


document_content_description = "Tóm tắt ngắn gọn về một bộ phim"



retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
)


### custom filter

In [9]:
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt,
)

prompt = get_query_constructor_prompt(
    document_content_description,
    metadata_field_info,
)
output_parser = StructuredQueryOutputParser.from_components()
query_constructor = prompt | llm | output_parser

In [14]:
print(prompt.format(query="dummy question"))

Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (eq | ne | gt | gte | lt | lte | contain | like | in | nin): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, statement2, ...)`:
- `op` (and | or | not

In [16]:
import json 

file_path = 'header.jsonl'
with open(file_path, 'r', encoding='utf-8') as file:
    data = []
    for line in file: 
        entry = json.loads(line.strip())
        data.append(entry)
        
for item in data:
    print(item)

{'Header 1': '2. Đối tượng đào tạo, ngành đào tạo', 'Header 2': '2.1. Đối tượng đào tạo trình độ đại học'}
{'Header 1': '2. Đối tượng đào tạo, ngành đào tạo', 'Header 2': '2.2. Ngành đào tạo'}
{'Header 1': '2. Đối tượng đào tạo, ngành đào tạo', 'Header 2': '2.2. Ngành đào tạo'}
{'Header 1': 'Thông tin tuyển sinh năm 2021, 2022, 2023 của học viện kỹ thuật quân sự', 'Header 2': '4.1. Phương thức tuyển sinh năm 2021, 2022, 2023'}
{'Header 1': 'Thông tin tuyển sinh năm 2021, 2022, 2023 của học viện kỹ thuật quân sự', 'Header 2': '4.2. Điểm trúng tuyển năm 2021, 2022, 2023'}
{'Header 1': 'CƠ HỘI HỌC TẬP VÀ ĐIỂM MỚI TUYỂN SINH NĂM 2024 CỦA HỌC VIỆN KỸ THUẬT QUÂN SỰ', 'Header 2': 'I. CƠ HỘI HỌC TẬP TẠI HỌC VIỆN KỸ THUẬT QUÂN SỰ'}
{'Header 1': 'CƠ HỘI HỌC TẬP VÀ ĐIỂM MỚI TUYỂN SINH NĂM 2024 CỦA HỌC VIỆN KỸ THUẬT QUÂN SỰ', 'Header 2': 'I. CƠ HỘI HỌC TẬP TẠI HỌC VIỆN KỸ THUẬT QUÂN SỰ'}
{'Header 1': 'CƠ HỘI HỌC TẬP VÀ ĐIỂM MỚI TUYỂN SINH NĂM 2024 CỦA HỌC VIỆN KỸ THUẬT QUÂN SỰ', 'Header 2': 'I. CƠ

In [19]:
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import JsonOutputParser

llm = ChatGroq(model_name="llama3-70b-8192", temperature=0.1, api_key=os.getenv('llm_api_1'))

system_prompt = f"""bạn là một trợ lý ảo hỗ trợ người dùng phân tích câu hỏi tìm ra header mà người dùng cần. Phản hồi bằng tiếng việt"""
prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)
chain = prompt | llm 
chain.invoke({"input": "điểm chuẩn vào học viện kỹ thuật quân sự"})

AIMessage(content='Based on your question, I think the header that you need is:\n\n**"Điểm chuẩn vào Học viện Kỹ thuật Quân sự"**\n\nWhich can be translated to:\n\n**"Admission scores for the Military Technical Academy"**\n\nIs that correct?', response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 57, 'total_tokens': 109, 'completion_time': 0.163513053, 'prompt_time': 0.005446394, 'queue_time': 0.065985441, 'total_time': 0.168959447}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_753a4aecf6', 'finish_reason': 'stop', 'logprobs': None}, id='run-aca05405-6406-45ab-adc0-3d9339134823-0', usage_metadata={'input_tokens': 57, 'output_tokens': 52, 'total_tokens': 109})

In [ ]:
from langchain import PromptTemplate, FewShotPromptTemplate


examples = [{
    "question": "Who was Albert Einstein and what is he best known for?",
    "answer": "He was a German-born theoretical physicist, widely acknowledged to be one of the greatest and most influential physicists of all time. He was best known for developing the theory of relativity, he also made important contributions to the development of the theory of quantum mechanics.",
    "sentences": "He was a German-born theoretical physicist, widely acknowledged to be one of the greatest and most influential physicists of all time",
    "analysis": '''
            Albert Einstein was a German-born theoretical physicist.,
            Albert Einstein is recognized as one of the greatest and most influential physicists of all time.,
'''
}]


# Định nghĩa mẫu cho mỗi ví dụ
example_template = """
Examples :\n
<Input>
Header 1: {header1}
Header 2: {header2}
Header 3: {header3}
query: {query}
</Input>\n\n
<Output>
Analysis: {analysis}
</Output>
"""

example_prompt = PromptTemplate(
    input_variables=["header1", "header2", "header3", "query"],
    template=example_template
)

# Định nghĩa prompt chính
prefix = "You are a virtual assistant who helps users find the corresponding paragraph based on the user's question. Response in Vietnamese."

suffix = """
Header 1: {header1}
Header 2: {header2}
Header 3: {header3}
query: {query}
analysis:
"""

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["header1", "header2", "header3", "query"],
    example_separator="\n\n"
)

In [ ]:
new_example =   {
        "question": "Dụng cụ trong ngành tài nguyên môi trường là gì?",
        "answer": "Dụng cụ trong ngành tài nguyên môi trường là loại tài sản không đủ tiêu chuẩn về tài sản cố định theo quy định hiện hành của nhà nước, mà người lao động sử dụng để tác động, biến đổi vật liệu thành sản phẩm. Ví dụ: kìm, búa, cờ lê, quần áo bảo hộ và các dụng cụ khác tương tự.",
        "sentences": "Trong Thông tư này các từ ngữ dưới đây được hiểu như sau:\n1. Định mức kinh tế - kỹ thuật (sau đây gọi tắt là định mức): Là mức hao phí cần thiết về lao động về nguyên, nhiên vật liệu, máy móc thiết bị, dụng cụ và phương tiện để hoàn thành một đơn vị sản phẩm (hoặc một khối lượng công việc nhất định), trong một điều kiện cụ thể của các hoạt động điều tra cơ bản trong các lĩnh vực thuộc phạm vi quản lý nhà nước của Bộ.\n2. Dụng cụ là loại tài sản không đủ tiêu chuẩn về tài sản cố định theo quy định hiện hành của nhà nước mà người lao động sử dụng để tác động, biến đổi vật liệu thành sản phẩm (kìm, búa, cờ lê, quần áo bảo hộ và các dụng cụ khác tương tự).\n3. Vật liệu là đầu vào trong một quá trình sản xuất hoặc thực hiện nhiệm vụ chuyên môn cụ thể, được người lao động tác động, biến đổi hoàn toàn để thành sản phẩm theo yêu cầu đặt ra.\n4. Máy móc thiết bị là công cụ lao động thuộc tài sản cố định hữu hình và tài sản cố định vô hình theo tiêu chuẩn quy định hiện hành của nhà nước về tài sản cố định (không bao gồm nhà xưởng và quyền sử dụng đất) mà người lao động sử dụng để tác động, biến đổi vật liệu thành sản phẩm."
    }

In [ ]:
prompt = few_shot_prompt_template.format(
    question=new_example["question"],
    answer=new_example["answer"],
    sentences=new_example["sentences"]
)
# ai_msg = llm.invoke(prompt)
# ai_msg

print(prompt.format(**examples[0]))


In [ ]:
out=llm.invoke(prompt)
out.content